In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import os
from zoneinfo import ZoneInfo
from typing import Iterable, Union, Tuple, List

from frequenz.data.microgrid import component_data
from frequenz.data.microgrid.config import MicrogridConfig
import frequenz.lib.notebooks.reporting.plotter as pl
from frequenz.client.common.metric import Metric        
from frequenz.client.reporting import ReportingApiClient 
from frequenz.lib.notebooks.reporting.utils.column_mapper import ColumnMapper
from frequenz.lib.notebooks.reporting.utils.helpers import set_date_to_midnight, add_energy_flows
from frequenz.lib.notebooks.reporting.data_processing import create_energy_report_df
from frequenz.lib.notebooks.reporting.utils.reporting_nb_functions import build_overview_df, compute_energy_summary, aggregate_metrics, build_component_analysis, assemble_component_analysis
from frequenz.lib.notebooks.reporting.plotter import plot_time_series

from IPython.display import display, Markdown

import logging
logging.basicConfig()
logging.getLogger("frequenz.lib.notebooks").setLevel(logging.WARNING)

pv_analyse_df, bat_analyse_df, chp_analyse_df, wind_analyse_df, ev_analyse_df = (pd.DataFrame() for _ in range(5))
ev_prod_filter_sum, wind_prod_filter_sum, chp_prod_filter_sum, bat_prod_filter_sum, pv_prod_filter_sum, pv_production_sum, net_site_consumption_sum, grid_consumption_sum, pv_feed_in_sum, peak, pv_self_consumption_sum, pv_self_consumption_share, pv_total_consumption_share = (0 for _ in range(13))

import warnings
warnings.filterwarnings("ignore")

In [2]:
color_dict = {
    "PV": "rgba(255,224,51,1)",
    "PV-Erzeugung": "rgba(255,224,51,1)",
    "PV-Erzeugung [kWh]": "rgba(255,224,51,1)",
    "PV-Erzeugung [kWh] Sum": "rgba(255,224,51,1)",
    "Wind": "rgba(100,149,237,1)",
    "Wind-Erzeugung": "rgba(100,149,237,1)",
    "Wind-Erzeugung [kWh] Sum": "rgba(100,149,237,1)",
    "BHKW": "rgba(255,140,0,1)",
    "BHKW-Erzeugung": "rgba(255,140,0,1)",
    "BHKW-Erzeugung [kWh] Sum": "rgba(255,140,0,1)",
    "Netto Gesamtverbrauch": "rgba(70,70,70,1)",
    "MID Gesamtverbrauch": "rgba(70,70,70,1)",
    "Batterie": "rgba(146,219,68,1)",
    "Netzbezug": "rgba(231,76,60,1)",
}

europe_timezones = [
    "Europe/Berlin",
    "Europe/Vienna",
    "Europe/Zurich",
    "Europe/London",
]

# Reporting Notebook

⚠️ **Wichtig:**
Um die Anweisungen anzuzeigen (falls nicht bereits angezeigt), müssen Sie **die nächste Zelle ausführen**.

▶️ **Wie führe ich die nächste Zelle aus?**
--> Drücken Sie die **Run** ▶️-Taste oben.


In [3]:
%%html
<style>
  details {
    background: #f8f9fa;
    border-radius: 6px;
    padding: 12px;
    margin-bottom: 12px;
    border: 1px solid #ddd;
  }

  summary {
    font-size: 18px;
    font-weight: bold;
    cursor: pointer;
    padding: 6px;
    border-radius: 4px;
  }

  summary:hover {
    color: #007bff;
  }

  p {
    font-size: 15px;
    line-height: 1.6;
  }
  
  ul {
    margin-left: 20px;
    padding-left: 15px;
    font-size: 15px;
  }
  
  li {
    margin-bottom: 8px;
  }
  
  /* Style checked items */
  .checked {
    text-decoration: line-through;
    color: gray;
  }
</style>

<details>
  <summary>🇩🇪 <strong>Deutsche Anleitung (Klicken zum Anzeigen)</strong></summary>

  <p><strong>🛠️ Anleitung</strong></p>
  <p>Das Notebook benötigt einige Einstellungen, bevor es zum ersten Mal ausgeführt werden kann.</p>

  <ul>
    <li><input type="checkbox" onclick="toggleStrike(this)"> ✅ <strong>Neueste Paketversion sicherstellen:</strong> Es wird empfohlen, die neuesten Version der benötigten Pakete zu installieren. Alle Versionen finden Sie hier: <a href="https://pypi.org/project/frequenz-lib-notebooks/#history">frequenz-lib-notebooks</a>, <a href="https://pypi.org/project/frequenz-client-reporting/#history">frequenz-client-reporting</a>. Ersetzen Sie bei der Installation die vordefinierte Version durch die gewünschte.</li>
    <li><input type="checkbox" onclick="toggleStrike(this)"> 🛠️ <strong>Führen Sie die erste Code-Zelle aus:</strong> Klicken Sie auf das <strong><span style="color: blue;">blaue</span> Dreieck</strong>, um Pakete zu installieren. Danach verschieben Sie sie zu <strong>"requirements.txt"</strong> (weitere Informationen finden Sie im Dokument <em>"Anleitung: Deepnote Projekt erstellen und Notebook importieren"</em>, Abschnitt <em>"5. Requirements installieren"</em>).</li>
    <li><input type="checkbox" onclick="toggleStrike(this)"> 📦 <strong>Importieren Sie die Bibliotheken:</strong> Führen Sie die nächsten zwei Code-Zellen aus, um die notwendigen Bibliotheken zu importieren und die Datei <code>microgrids.toml</code> zu laden. ⏳ <em>Warten Sie, bis die Ausführung abgeschlossen ist.</em></li>
    <li><input type="checkbox" onclick="toggleStrike(this)"> 🎯 <strong>Konfigurieren Sie die Eingaben:</strong> Füllen Sie die erforderlichen Felder unter <strong>"Input"</strong> aus. </li>
    <li><input type="checkbox" onclick="toggleStrike(this)"> 🚀 <strong>Notebook ausführen:</strong> Klicken Sie auf den <strong><span style="color: blue;">"Start"</span>-Button</strong>, sobald alle Eingaben ausgefüllt sind.</li>
  </ul>

</details>

<script>
  function toggleStrike(checkbox) {
    if (checkbox.checked) {
      checkbox.parentElement.classList.add("checked");
    } else {
      checkbox.parentElement.classList.remove("checked");
    }
  }
</script>

# Einleitung

In diesem Notebook können Sie die Daten aus der Reporting API analysieren und visualisieren. Sie haben die Möglichkeit, den gewünschten Zeitraum und die Auflösung der zu analysierenden Daten festzulegen.
Die Analyse umfasst wichtige Kennzahlen zur Stromgewinnung aus Photovoltaikanlagen (PV), zur Nutzung von Batteriespeichern sowie zum Netzanschluss. Die Ergebnisse werden in anschaulichen Charts dargestellt, um Ihnen einen klaren Überblick über die Energieflüsse zu ermöglichen.
Wählen Sie einfach Ihre gewünschten Parameter aus, um tiefere Einblicke in Ihre Energieerzeugung und -nutzung zu erhalten.

# Eingaben

In [4]:
directory = "toml_directory/"
configs = MicrogridConfig.load_configs(microgrid_config_dir=directory)
available_microgrids = sorted(list(int(x) for x in configs.keys()))

Um das Notebook nutzen zu können müssen zunächst die API Anmeldeinformationen hinterlegt werden (Anleitung). Die API Anmeldeinformationen finden Sie im Kuiper-Portal. Dort finden Sie ebenfalls ihre Microgrid ID, die Meter PQ ID, die Meter Batterie IDs sowie die Meter PV IDs. Bitte geben Sie Ihre Komponenten IDs unten ein, wählen sie ein Start- und Enddatum, wählen sie eine Auflösung und klicken Sie auf Start, um das Notebook auszuführen.
Eine Anleitung zum Vorgehen finden Sie hier.

In [5]:
microgrid_id = '13'

Zeitrahmen

In [6]:

from dateutil.parser import parse as _deepnote_parse
start_date = _deepnote_parse('2025-10-01T00:00:00.000Z').date()


In [7]:

from dateutil.parser import parse as _deepnote_parse
end_date = _deepnote_parse('2025-10-14T00:00:00.000Z').date()


In [8]:
timezone = 'Europe/Berlin'

In [9]:
resolution = '15'

# 

# 

# 

In [10]:
mapper = ColumnMapper.from_default()
start_date = set_date_to_midnight(start_date, "Europe/Berlin")
end_date = set_date_to_midnight(end_date, "Europe/Berlin")
processed_resolution = timedelta(seconds=int(resolution) * 60)
timestep_hours = int(resolution) / 60
microgrid_id = int(microgrid_id)
print(f"Microgrid ID: {microgrid_id}")
print(f"Start Date: {start_date}")
print(f"End Date: {end_date}")
print(f"Resolution: {processed_resolution}")

Microgrid ID: 13
Start Date: 2025-10-01 00:00:00+02:00
End Date: 2025-10-14 00:00:00+02:00
Resolution: 0:15:00


## Datenimport

In [11]:
mg_config = MicrogridConfig.load_configs(
    microgrid_config_dir=directory
)

mdata = component_data.MicrogridData(
    server_url=os.environ["REPORTING_API_URL"],
    auth_key=os.environ["REPORTING_API_KEY"],
    sign_secret=os.environ["REPORTING_API_SECRET"],
    microgrid_configs=mg_config,
) 

mids = [i.replace("iot", "") for i in mdata.microgrid_ids] 

mcfg = mdata.microgrid_configs[str(microgrid_id)]
ctypes = mcfg.component_types()
component_types = ctypes

df = await mdata.ac_active_power(
    microgrid_id=microgrid_id, 
    component_types=component_types,
    start=start_date,
    end=end_date,
    resampling_period=processed_resolution,
    keep_components=True,
    splits=True,
)

# Remove the component_types where the meters are not working
present = [c for c in component_types if c in df.columns]

# Optionally, see which were missing
missing = [c for c in component_types if c not in df.columns]
if missing:
    print(f"Skipping missing components (no column in df): {missing}")

# Now drop components whose meters sum to zero
component_types = [
    c for c in present
    if pd.to_numeric(df[c], errors="coerce").fillna(0).sum() != 0
]


print(f"Received {df.shape[0]} rows and {df.shape[1]} columns")
# df.to_csv('raw_df.csv')

Received 1247 rows and 63 columns


In [12]:
energy_report_df = create_energy_report_df(df, component_types, mcfg, mapper=mapper, tz_name=timezone, assume_tz="UTC")
energy_metrics_dict = aggregate_metrics(energy_report_df=energy_report_df, resolution=processed_resolution)

(
    pv_production_sum,
    chp_production_sum,
    wind_production_sum,
    prod_self_consumption_sum,
    prod_bat_sum,
    grid_feed_in_sum,
    grid_import_sum,
    mid_consumption_sum,
    total_production_sum,
    prod_self_consumption_share,
    peak,
    peak_date
) = energy_metrics_dict.values()

# Übersicht

In [13]:
print(f'{datetime.strftime(start_date.astimezone(ZoneInfo("CET")).date(), "%d.%m.%Y")} - {datetime.strftime(end_date.astimezone(ZoneInfo("CET")).date(), "%d.%m.%Y")}')

01.10.2025 - 14.10.2025


### Stromverbrauch

In [14]:

def __deepnote_big_number__():
    import json
    import jinja2
    from jinja2 import meta

    def render_template(template):
        parsed_content = jinja2.Environment().parse(template)

        required_variables = meta.find_undeclared_variables(parsed_content)

        context = {
            variable_name: globals().get(variable_name)
            for variable_name in required_variables
        }

        result = jinja2.Environment().from_string(template).render(context)

        return result

    rendered_title = render_template("Brutto Stromverbrauch kWh")
    rendered_comparison_title = render_template("")

    return json.dumps({
        "comparisonTitle": rendered_comparison_title,
        "comparisonValue": "",
        "title": rendered_title,
        "value": f"{mid_consumption_sum}"
    })

__deepnote_big_number__()


'{"comparisonTitle": "", "comparisonValue": "", "title": "Brutto Stromverbrauch kWh", "value": "52728.23614648437"}'

# 

# 

# 

### Netz

In [15]:

def __deepnote_big_number__():
    import json
    import jinja2
    from jinja2 import meta

    def render_template(template):
        parsed_content = jinja2.Environment().parse(template)

        required_variables = meta.find_undeclared_variables(parsed_content)

        context = {
            variable_name: globals().get(variable_name)
            for variable_name in required_variables
        }

        result = jinja2.Environment().from_string(template).render(context)

        return result

    rendered_title = render_template("Bezug in kWh")
    rendered_comparison_title = render_template("")

    return json.dumps({
        "comparisonTitle": rendered_comparison_title,
        "comparisonValue": "",
        "title": rendered_title,
        "value": f"{grid_import_sum}"
    })

__deepnote_big_number__()


'{"comparisonTitle": "", "comparisonValue": "", "title": "Bezug in kWh", "value": "9752.259956350328"}'

In [16]:

def __deepnote_big_number__():
    import json
    import jinja2
    from jinja2 import meta

    def render_template(template):
        parsed_content = jinja2.Environment().parse(template)

        required_variables = meta.find_undeclared_variables(parsed_content)

        context = {
            variable_name: globals().get(variable_name)
            for variable_name in required_variables
        }

        result = jinja2.Environment().from_string(template).render(context)

        return result

    rendered_title = render_template("Einspeisung in kWh")
    rendered_comparison_title = render_template("")

    return json.dumps({
        "comparisonTitle": rendered_comparison_title,
        "comparisonValue": "",
        "title": rendered_title,
        "value": f"{grid_feed_in_sum}"
    })

__deepnote_big_number__()


'{"comparisonTitle": "", "comparisonValue": "", "title": "Einspeisung in kWh", "value": "18.100957000732418"}'

In [17]:

def __deepnote_big_number__():
    import json
    import jinja2
    from jinja2 import meta

    def render_template(template):
        parsed_content = jinja2.Environment().parse(template)

        required_variables = meta.find_undeclared_variables(parsed_content)

        context = {
            variable_name: globals().get(variable_name)
            for variable_name in required_variables
        }

        result = jinja2.Environment().from_string(template).render(context)

        return result

    rendered_title = render_template("Lastspitze in kW {{peak_date}}")
    rendered_comparison_title = render_template("")

    return json.dumps({
        "comparisonTitle": rendered_comparison_title,
        "comparisonValue": "",
        "title": rendered_title,
        "value": f"{peak}"
    })

__deepnote_big_number__()


'{"comparisonTitle": "", "comparisonValue": "", "title": "Lastspitze in kW 01.01.1970", "value": "137.742796875"}'

# 

### Gesamterzeugung

In [18]:

def __deepnote_big_number__():
    import json
    import jinja2
    from jinja2 import meta

    def render_template(template):
        parsed_content = jinja2.Environment().parse(template)

        required_variables = meta.find_undeclared_variables(parsed_content)

        context = {
            variable_name: globals().get(variable_name)
            for variable_name in required_variables
        }

        result = jinja2.Environment().from_string(template).render(context)

        return result

    rendered_title = render_template("Gesamterzeugung in kWh")
    rendered_comparison_title = render_template("")

    return json.dumps({
        "comparisonTitle": rendered_comparison_title,
        "comparisonValue": "",
        "title": rendered_title,
        "value": f"{total_production_sum}"
    })

__deepnote_big_number__()


'{"comparisonTitle": "", "comparisonValue": "", "title": "Gesamterzeugung in kWh", "value": "43514.21395677591"}'

In [19]:

def __deepnote_big_number__():
    import json
    import jinja2
    from jinja2 import meta

    def render_template(template):
        parsed_content = jinja2.Environment().parse(template)

        required_variables = meta.find_undeclared_variables(parsed_content)

        context = {
            variable_name: globals().get(variable_name)
            for variable_name in required_variables
        }

        result = jinja2.Environment().from_string(template).render(context)

        return result

    rendered_title = render_template("PV Gesamterzeugung in kWh")
    rendered_comparison_title = render_template("")

    return json.dumps({
        "comparisonTitle": rendered_comparison_title,
        "comparisonValue": "",
        "title": rendered_title,
        "value": f"{pv_production_sum}"
    })

__deepnote_big_number__()


'{"comparisonTitle": "", "comparisonValue": "", "title": "PV Gesamterzeugung in kWh", "value": "6294.780380604029"}'

In [20]:

def __deepnote_big_number__():
    import json
    import jinja2
    from jinja2 import meta

    def render_template(template):
        parsed_content = jinja2.Environment().parse(template)

        required_variables = meta.find_undeclared_variables(parsed_content)

        context = {
            variable_name: globals().get(variable_name)
            for variable_name in required_variables
        }

        result = jinja2.Environment().from_string(template).render(context)

        return result

    rendered_title = render_template("BHKW Gesamterzeugung in kWh")
    rendered_comparison_title = render_template("")

    return json.dumps({
        "comparisonTitle": rendered_comparison_title,
        "comparisonValue": "",
        "title": rendered_title,
        "value": f"{chp_production_sum}"
    })

__deepnote_big_number__()


'{"comparisonTitle": "", "comparisonValue": "", "title": "BHKW Gesamterzeugung in kWh", "value": "37219.433576171876"}'

In [21]:

def __deepnote_big_number__():
    import json
    import jinja2
    from jinja2 import meta

    def render_template(template):
        parsed_content = jinja2.Environment().parse(template)

        required_variables = meta.find_undeclared_variables(parsed_content)

        context = {
            variable_name: globals().get(variable_name)
            for variable_name in required_variables
        }

        result = jinja2.Environment().from_string(template).render(context)

        return result

    rendered_title = render_template("Wind Gesamterzeugung in kWh")
    rendered_comparison_title = render_template("")

    return json.dumps({
        "comparisonTitle": rendered_comparison_title,
        "comparisonValue": "",
        "title": rendered_title,
        "value": f"{wind_production_sum}"
    })

__deepnote_big_number__()


'{"comparisonTitle": "", "comparisonValue": "", "title": "Wind Gesamterzeugung in kWh", "value": "0.0"}'

### Eigenverbrauch

In [22]:

def __deepnote_big_number__():
    import json
    import jinja2
    from jinja2 import meta

    def render_template(template):
        parsed_content = jinja2.Environment().parse(template)

        required_variables = meta.find_undeclared_variables(parsed_content)

        context = {
            variable_name: globals().get(variable_name)
            for variable_name in required_variables
        }

        result = jinja2.Environment().from_string(template).render(context)

        return result

    rendered_title = render_template("Eigenverbrauch in kWh")
    rendered_comparison_title = render_template("")

    return json.dumps({
        "comparisonTitle": rendered_comparison_title,
        "comparisonValue": "",
        "title": rendered_title,
        "value": f"{prod_self_consumption_sum}"
    })

__deepnote_big_number__()


'{"comparisonTitle": "", "comparisonValue": "", "title": "Eigenverbrauch in kWh", "value": "41985.92115062356"}'

In [23]:

def __deepnote_big_number__():
    import json
    import jinja2
    from jinja2 import meta

    def render_template(template):
        parsed_content = jinja2.Environment().parse(template)

        required_variables = meta.find_undeclared_variables(parsed_content)

        context = {
            variable_name: globals().get(variable_name)
            for variable_name in required_variables
        }

        result = jinja2.Environment().from_string(template).render(context)

        return result

    rendered_title = render_template("Eigenverbrauchsanteil in kWh")
    rendered_comparison_title = render_template("")

    return json.dumps({
        "comparisonTitle": rendered_comparison_title,
        "comparisonValue": "",
        "title": rendered_title,
        "value": f"{prod_self_consumption_share}"
    })

__deepnote_big_number__()


'{"comparisonTitle": "", "comparisonValue": "", "title": "Eigenverbrauchsanteil in kWh", "value": "0.7962701622330477"}'

# 

# 

Lastgang im zeitlichen Verlauf: 
Hier werden die PV-Gesamtleistung, der Verbrauch (Last), der Netzbezug, die (PV) Einspeisung und die Batterie-Gesamtleistung im ausgewählten Zeitraum dargestellt.

In [24]:
overview_df = build_overview_df(energy_report_df, component_types=component_types)
overview_df = mapper.to_display(overview_df)
overview_rename_dict = {'grid_import': "Netzbezug", 'grid_feed_in': "Netzeinspeisung", 'pv_asset_production': "PV-Erzeugung", 'chp_asset_production': "BHKW-Erzeugung", "wind_asset_production": "Wind-Erzeugung", 'Batterie Durchsatz': "Batterie"}
overview_df = overview_df.rename(columns=overview_rename_dict)
plot_order = ["Zeitpunkt", 'MID Gesamtverbrauch', 'Netzbezug', 'Netzeinspeisung', 'PV-Erzeugung', 'BHKW-Erzeugung', 'Wind-Erzeugung', 'Batterie']
# fig = plot_time_series(overview_df, time_col="Zeitpunkt", cols=None, title="Lastgang Übersicht", color_dict=color_dict, plot_order=plot_order)
fig = plot_time_series(overview_df, time_col="Zeitpunkt", cols=None, title="Lastgang Übersicht", color_dict=color_dict, fill_cols=overview_df.columns.tolist(), plot_order=plot_order)

## Energiebezug
Hier sehen Sie das Verhältnis von Eigenverbrauch und Netzbezug in dem von Ihnen gewählten Zeitraum an.

In [25]:
energy_summary_df = compute_energy_summary(energy_report_df, resolution=processed_resolution)
energy_summary_df["Energy Source"] = energy_summary_df["Energy Source"].replace({'CHP': 'BHKW', 'Grid Import': 'Netzbezug'})
energy_summary_df = energy_summary_df.rename(columns={"Energy Source": "Energiebezug", "Energy [kWh]": "Energie [kWh]"})
# energy_summary_df

In [26]:
energy_pie_plot = pl.plot_energy_pie_chart(energy_summary_df, color_dict=color_dict)
# energy_pie_plot.show()

## PV

PV-Leistung 
Hier sehen Sie sowohl die kumulierte Leistung der PV-Anlagen als auch die gestapelten Einzelleistungen der jeweiligen PV-Anlagen. Die Netzanschluss-Kurve zeigt an, wie viel Stromüberschüsse ins Netz eingespeist wurden (Kurve im negativen Bereich). Beziehungsweise wieviel Strom vom Netz bezogen wurde (Kurve im positiven Bereich).
Um einzelne PV-Anlagen zu filtern muss zunächst "Alle" aus dem Filter entfernt und dann die gewünschten PV IDs ausgewählt werden.

In [27]:
pv_filter_options = []
pv_columns = [c for c in energy_report_df.columns if "PV #" in c]
if 'pv' in component_types:
    pv_grid_filter_options = ["PV und Netzanschluss", "Nur PV", "Nur Netzanschluss"]
    pv_filter_options = ['Alle'] + [pv[3:] for pv in pv_columns]

In [28]:
available_pv_units = ['Alle']

Filter_PV = False

In [30]:
if available_pv_units or Filter_PV:
    pv_analyse_df, pv_prod_filter_sum, pv_filter_text = assemble_component_analysis(
        component_filter=available_pv_units,
        component_key="pv",
        component_types=component_types,
        energy_report_df=energy_report_df,
        timestep_hours=timestep_hours,
        mapper=mapper,
        component_label="PV",
        value_col_name="pv_asset_production",
        invert_sign=True,
        trunc_values=True,
    )
    pv_analyse_df = pv_analyse_df.rename(columns={"pv_asset_production": "PV-Erzeugung [kWh]"})
else:
    pv_analyse_df = pd.DataFrame()    
# pv_analyse_df

In [31]:

def __deepnote_big_number__():
    import json
    import jinja2
    from jinja2 import meta

    def render_template(template):
        parsed_content = jinja2.Environment().parse(template)

        required_variables = meta.find_undeclared_variables(parsed_content)

        context = {
            variable_name: globals().get(variable_name)
            for variable_name in required_variables
        }

        result = jinja2.Environment().from_string(template).render(context)

        return result

    rendered_title = render_template("PV Gesamterzeugung ({{pv_filter_text}}) in kWh")
    rendered_comparison_title = render_template("")

    return json.dumps({
        "comparisonTitle": rendered_comparison_title,
        "comparisonValue": "",
        "title": rendered_title,
        "value": f"{pv_prod_filter_sum}"
    })

__deepnote_big_number__()


'{"comparisonTitle": "", "comparisonValue": "", "title": "PV Gesamterzeugung (All) in kWh", "value": "6294.78"}'

In [32]:
if available_pv_units or Filter_PV:
    if "pv" in component_types:
        fig = plot_time_series(
            pv_analyse_df,
            time_col="Zeitpunkt",
            title="PV Leistung",
            yaxis_title="kWh",
            xaxis_title="Zeitpunkt",
            legend_title=None,
            long_format_flag=True,
            category_col="PV",
            value_col="PV-Erzeugung [kWh]",
            color_dict=color_dict
        )
        # fig.show()

## Batterie

Batterieleistungskurve 
Die Batterieleistungs-Kurve zeigt kumuliert für alle Batterien an, wie viel Strom aus der Batterie bezogen wurde (Kurve im negativen Bereich). Beziehungsweise wieviel Strom in die Batterie eingespeist wurde (Kurve im positiven Bereich). 
Um einzelne Batterien zu filtern muss zunächst "Alle" aus dem Filter entfernt und dann die gewünschten Batterie IDs ausgewählt werden.

In [33]:
battery_filter_options = []
if 'battery' in component_types:
    battery_filter_options = ['Alle'] + [f'#{i}' for i in mcfg.component_type_ids('battery')]

In [34]:
available_battery_units = ['#263','#260','#275','Alle']

Filter_Battery = False

In [36]:
if available_battery_units or Filter_Battery:
    bat_analyse_df, bat_prod_filter_sum, bat_filter_text = assemble_component_analysis(
        component_filter=available_battery_units,
        component_key="battery",
        component_types=component_types,
        energy_report_df=energy_report_df,
        timestep_hours=timestep_hours,
        mapper=mapper,
        component_label="Battery",
        value_col_name="Batterie Durchsatz",
    )
    bat_analyse_df = bat_analyse_df.rename(columns={"Batterie Durchsatz": "Batterie Energie [kWh]"})

else:
    bat_analyse_df = pd.DataFrame()    

# bat_analyse_df

In [37]:

def __deepnote_big_number__():
    import json
    import jinja2
    from jinja2 import meta

    def render_template(template):
        parsed_content = jinja2.Environment().parse(template)

        required_variables = meta.find_undeclared_variables(parsed_content)

        context = {
            variable_name: globals().get(variable_name)
            for variable_name in required_variables
        }

        result = jinja2.Environment().from_string(template).render(context)

        return result

    rendered_title = render_template("Battery Energie ({{bat_filter_text}}) in kWh")
    rendered_comparison_title = render_template("")

    return json.dumps({
        "comparisonTitle": rendered_comparison_title,
        "comparisonValue": "",
        "title": rendered_title,
        "value": f"{bat_prod_filter_sum}"
    })

__deepnote_big_number__()


'{"comparisonTitle": "", "comparisonValue": "", "title": "Battery Energie (All) in kWh", "value": "425.235"}'

In [38]:
if available_battery_units or Filter_Battery:
    if "battery" in {str(x).lower() for x in component_types}:
        fig = plot_time_series(
            bat_analyse_df,
            time_col="Zeitpunkt",
            title="Batterie Energie [kWh]",
            yaxis_title="kWh",
            xaxis_title="Zeitpunkt",
            legend_title=None,
            long_format_flag=True,
            category_col="Battery",
            value_col="Batterie Energie [kWh]",
            color_dict=color_dict
        )
        # fig.show()

# BHKW

In [39]:
chp_filter_options = []
if 'chp' in component_types:
    chp_filter_options = ['Alle'] + [f'#{i}' for i in mcfg.component_type_ids('chp')]

In [40]:
available_chp_units = ['Alle']

Filter_BHKW = False

In [42]:
if available_chp_units or Filter_BHKW:
    chp_analyse_df, chp_prod_filter_sum, chp_filter_text = assemble_component_analysis(
        component_filter=available_chp_units,
        component_key="chp",
        component_types=component_types,
        energy_report_df=energy_report_df,
        timestep_hours=timestep_hours,
        mapper=mapper,
        component_label="CHP",
        value_col_name="chp_asset_production",
        invert_sign=True,
        trunc_values=True,
    )
    chp_analyse_df = chp_analyse_df.rename(columns={"chp_asset_production": "BHKW-Erzeugung [kWh]", "CHP": "BHKW"})

else:
    chp_analyse_df = pd.DataFrame()

# chp_analyse_df

In [43]:

def __deepnote_big_number__():
    import json
    import jinja2
    from jinja2 import meta

    def render_template(template):
        parsed_content = jinja2.Environment().parse(template)

        required_variables = meta.find_undeclared_variables(parsed_content)

        context = {
            variable_name: globals().get(variable_name)
            for variable_name in required_variables
        }

        result = jinja2.Environment().from_string(template).render(context)

        return result

    rendered_title = render_template("BHKW Gesamterzeugung ({{chp_filter_text}}) in kWh")
    rendered_comparison_title = render_template("")

    return json.dumps({
        "comparisonTitle": rendered_comparison_title,
        "comparisonValue": "",
        "title": rendered_title,
        "value": f"{chp_prod_filter_sum}"
    })

__deepnote_big_number__()


'{"comparisonTitle": "", "comparisonValue": "", "title": "BHKW Gesamterzeugung (All) in kWh", "value": "37219.434"}'

In [44]:
if available_chp_units or Filter_BHKW:
    if "chp" in {str(x).lower() for x in component_types}:
        fig = plot_time_series(
            chp_analyse_df,
            time_col="Zeitpunkt",
            title="BHKW Erzeugung",
            yaxis_title="kWh",
            xaxis_title="Zeitpunkt",
            legend_title=None,
            long_format_flag=True,
            category_col="BHKW",
            value_col="BHKW-Erzeugung [kWh]",
            color_dict=color_dict
        )
        # fig.show()

# Wind

In [45]:
wind_filter_options = []
if 'wind' in component_types:
    wind_filter_options = ['Alle'] + [f'#{i}' for i in mcfg.component_type_ids('wind')]

In [46]:
available_wind_units = []

Filter_Wind = False

In [48]:
if available_wind_units or Filter_Wind:
    wind_analyse_df, wind_prod_filter_sum, wind_filter_text = assemble_component_analysis(
        component_filter=available_wind_units,
        component_key="wind",
        component_types=component_types,
        energy_report_df=energy_report_df,
        timestep_hours=timestep_hours,
        mapper=mapper,
        component_label="Wind",
        value_col_name="wind_asset_production",
        invert_sign=True,
        trunc_values=True,
    )
    wind_analyse_df = wind_analyse_df.rename(columns={"wind_asset_production": "Wind-Erzeugung [kWh]"})

# wind_analyse_df

In [49]:

def __deepnote_big_number__():
    import json
    import jinja2
    from jinja2 import meta

    def render_template(template):
        parsed_content = jinja2.Environment().parse(template)

        required_variables = meta.find_undeclared_variables(parsed_content)

        context = {
            variable_name: globals().get(variable_name)
            for variable_name in required_variables
        }

        result = jinja2.Environment().from_string(template).render(context)

        return result

    rendered_title = render_template("Wind Gesamterzeugung ({{wind_filter_text}}) in kWh")
    rendered_comparison_title = render_template("")

    return json.dumps({
        "comparisonTitle": rendered_comparison_title,
        "comparisonValue": "",
        "title": rendered_title,
        "value": f"{wind_prod_filter_sum}"
    })

__deepnote_big_number__()


'{"comparisonTitle": "", "comparisonValue": "", "title": "Wind Gesamterzeugung (None) in kWh", "value": "0"}'

In [50]:
if available_wind_units or Filter_Wind:
    if "wind" in {str(x).lower() for x in component_types}:
        fig = plot_time_series(
            wind_analyse_df,
            time_col="Zeitpunkt",
            title="Wind Erzeugung",
            yaxis_title="kWh",
            xaxis_title="Zeitpunkt",
            legend_title=None,
            long_format_flag=True,
            category_col="Wind",
            value_col="Wind-Erzeugung [kWh]",
            color_dict=color_dict
        )
        # fig.show()

# EV Charger

In [51]:
ev_filter_options = []
if 'ev' in component_types:
    ev_filter_options = ['Alle'] + [f'#{i}' for i in mcfg.component_type_ids('ev')]

In [52]:
available_ev_units = []

Filter_EV = False

In [54]:
if available_ev_units or Filter_EV:
    ev_analyse_df, ev_prod_filter_sum, ev_filter_text = assemble_component_analysis(
        component_filter=available_ev_units,
        component_key="ev",
        component_types=component_types,
        energy_report_df=energy_report_df,
        timestep_hours=timestep_hours,
        mapper=mapper,
        component_label="EV",
        value_col_name="ev_asset_production",
        invert_sign=True,
        trunc_values=True,
    )
    ev_analyse_df = ev_analyse_df.rename(columns={"ev_asset_production": "EV-Erzeugung [kWh]"})

# ev_analyse_df

In [55]:

def __deepnote_big_number__():
    import json
    import jinja2
    from jinja2 import meta

    def render_template(template):
        parsed_content = jinja2.Environment().parse(template)

        required_variables = meta.find_undeclared_variables(parsed_content)

        context = {
            variable_name: globals().get(variable_name)
            for variable_name in required_variables
        }

        result = jinja2.Environment().from_string(template).render(context)

        return result

    rendered_title = render_template("EV Gesamterzeugung ({{ev_filter_text}}) in kWh")
    rendered_comparison_title = render_template("")

    return json.dumps({
        "comparisonTitle": rendered_comparison_title,
        "comparisonValue": "",
        "title": rendered_title,
        "value": f"{ev_prod_filter_sum}"
    })

__deepnote_big_number__()


'{"comparisonTitle": "", "comparisonValue": "", "title": "EV Gesamterzeugung (None) in kWh", "value": "0"}'

In [56]:
if available_ev_units or Filter_EV:
    if "ev" in {str(x).lower() for x in component_types}:
        fig = plot_time_series(
            ev_analyse_df,
            time_col="Zeitpunkt",
            title="EV Erzeugung",
            yaxis_title="kWh",
            xaxis_title="Zeitpunkt",
            legend_title=None,
            long_format_flag=True,
            category_col="EV",
            value_col="EV-Erzeugung [kWh]",
            color_dict=color_dict
        )
        # fig.show()

## Daten

In [57]:
print(f'{datetime.strftime(start_date.astimezone(ZoneInfo("CET")).date(), "%d.%m.%Y")} - {datetime.strftime(end_date.astimezone(ZoneInfo("CET")).date(), "%d.%m.%Y")}')

01.10.2025 - 14.10.2025


### Daten exportieren

Die Daten werden unter angegebenem Dateinamen in den Deepnote Files gespeichert und können von dort heruntergeladen werden.
Mehr erfahren

In [58]:
output = ''

download_data = False

In [60]:
if output != "" and energy_report_df is not None and download_data:
    output_df = mapper.to_display(energy_report_df)
    if os.path.exists(output):
        print("Fehler: Die Daten wurden nicht exportiert. Es gibt bereits eine Datei mit diesem Namen.")
    elif output.endswith(".csv"):
        output_df['Zeitpunkt'] = output_df['Zeitpunkt'].dt.strftime("%d.%m.%Y %H:%M:%S")
        output_df.to_csv(output, index=False)
        print(f"Die Daten wurden erfolgreich in {output} exportiert.")
    elif output.endswith(".xlsx"):
        output_df['Zeitpunkt'] = output_df['Zeitpunkt'].dt.strftime("%d.%m.%Y %H:%M:%S")
        output_df.to_excel(output, index=False)
        print(f"Die Daten wurden erfolgreich in {output} exportiert.")
    else:
        print("Fehler: Die Daten wurden nicht exportiert. Output muss .csv oder .xlsx sein.")

else:
    output_df = pd.DataFrame()
output_df

Empty DataFrame
Columns: []
Index: []

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7c2a83fc-1e98-4f36-aef2-c01335f1a6a3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>